In [26]:
### basic libraries
import os
import itertools
from datetime import datetime
from json import dumps
import pandas as pd 
import numpy as np

### carto libraries
import shapefile ### not necessary ?
import geopandas as gp
#from shapely.geometry import Polygon
from shapely.geometry import shape, Point

### for plotting in Jupyter
%matplotlib inline
import matplotlib.pyplot as plt
#import seaborn as sns
plt.rcParams["figure.figsize"] = (10.0, 10.0)
#plt.style.use("bmh")
plt.style.use("ggplot")

#from app.scripts.topojson import topojson

from pyproj import Proj, transform ### not necessary ?

crs_lambert93 = 2154
crs_WSG84     = 4326
inCRS  = 'epsg:%s' %(crs_lambert93)
outCRS = 'epsg:%s' %(crs_WSG84)
inProj  = Proj(init=inCRS)  # proj in  : Lambert 93
outProj = Proj(init=outCRS) # proj out : WSG 84



In [27]:
### basic folders addresses and names

cwd = os.getcwd()

data_folder      = "app/static/data"

stats_folder     = "stats"
_web             = "_web" 
stats_web_folder = stats_folder + _web

stats_path     = os.path.join(cwd, data_folder, stats_folder)
stats_web_path = os.path.join(cwd, data_folder, stats_web_folder)


carto_folder     = "carto"
_web             = "_web"
carto_web_folder = carto_folder + _web

carto_path     = os.path.join(cwd, data_folder, carto_folder)
carto_web_path = os.path.join(cwd, data_folder, carto_web_folder)



print "-- cwd :"            , cwd
print "-- stats_web_path : ", stats_web_path
print "-- carto_web_path : ", carto_web_path

-- cwd : /Users/jpy/Dropbox/_FLASK/concours_pesticides
-- stats_web_path :  /Users/jpy/Dropbox/_FLASK/concours_pesticides/app/static/data/stats_web
-- carto_web_path :  /Users/jpy/Dropbox/_FLASK/concours_pesticides/app/static/data/carto_web


In [28]:
### read stations_web.csv from : data/stats_web

_sep_csv         = ";"
csv_encoding_web = "utf-8"

stations_filename = "stations_web.csv"
stations_web_filepath = os.path.join(stats_web_path, stations_filename)
print stations_web_filepath

df_stations_web = pd.read_csv( stations_web_filepath, sep=_sep_csv, encoding=csv_encoding_web )

df_stations_web.head(3)

/Users/jpy/Dropbox/_FLASK/concours_pesticides/app/static/data/stats_web/stations_web.csv


,NUM_DEP,NOM_COM,CD_ME_niv1_surf,CD_ME_v2,CD_STATION,INDEX_STATION,NUM_COM,codagence,ALTITUDE,PROFONDEUR_MAXI_POINT,...,fi_ma_2008,fi_ma_2009,fi_ma_2010,fi_ma_2011,fi_ma_2012,fi_ma_2013,fi_ma_2014,CD_STATION_,LAT_WSG84,LONG_WSG84
0,NaN,NaN,NaN,NaN,10261X0039/F3,31,NaN,AEAG,10.0,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10261X0039/F3,-1.363081,-5.983856
1,NaN,NaN,NaN,CG004,01688X0034/AVAL,33,NaN,AERM,235.0,NaN,...,NaN,oui,NaN,NaN,NaN,NaN,NaN,01688X0034/AVAL,-1.363081,-5.983856
2,NaN,NaN,NaN,CG004,01688X0039/F1,32,NaN,AERM,170.0,80,...,NaN,oui,NaN,NaN,NaN,NaN,NaN,01688X0039/F1,-1.363081,-5.983856


In [29]:
### create point geometries

geometry     = [ Point(xy) for xy in zip(df_stations_web.LAT_WSG84, df_stations_web.LONG_WSG84) ]

gdf_stations = gp.GeoDataFrame(df_stations_web, crs=crs_WSG84, geometry=geometry)

gdf_stations.tail(3)

,NUM_DEP,NOM_COM,CD_ME_niv1_surf,CD_ME_v2,CD_STATION,INDEX_STATION,NUM_COM,codagence,ALTITUDE,PROFONDEUR_MAXI_POINT,...,fi_ma_2009,fi_ma_2010,fi_ma_2011,fi_ma_2012,fi_ma_2013,fi_ma_2014,CD_STATION_,LAT_WSG84,LONG_WSG84,geometry
13036,95,VIENNE-EN-ARTHIES,HG107,NaN,01514X0029/P,13036,95656,AESN,100.0,6.5,...,oui,NaN,NaN,NaN,NaN,NaN,01514X0029/P,1.721380,49.052672,POINT (1.72137996602 49.0526718809)
13037,95,VILLERS-EN-ARTHIES,HG107,NaN,01514X0005/HY,13037,95676,AESN,100.0,NaN,...,oui,NaN,NaN,NaN,NaN,NaN,01514X0005/HY,1.744907,49.090419,POINT (1.74490703083 49.0904192425)
13038,95,WY-DIT-JOLI-VILLAGE,HG107,NaN,01521X0004/HY,13038,95690,AESN,110.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,01521X0004/HY,1.838831,49.098160,POINT (1.83883087182 49.0981601538)


In [30]:
#import fiona
#fiona.crs.from_epsg(4326)

In [31]:
### set projection CRS WSG84 

gdf_stations.crs = {
                    'init' : 'epsg:%s' %(crs_WSG84),
                    'no_defs': True
                   }

print gdf_stations.crs


{'init': 'epsg:4326', 'no_defs': True}


In [32]:
# work on copy
gdf_stations_json = gdf_stations.copy()

### set indexes :

gdf_stations_json.set_index([
                        #"NUM_DEP", 
                        "NUM_COM", 
                        #"NOM_COM",  
                        #"CD_ME_niv1_surf", 
                        #"CD_ME_v2", 
                        #"CD_STATION"
                       ], inplace=True, drop=False) 
#gdf_stations.set_index("CD_STATION", inplace=True) 
gdf_stations_json.sort_index(inplace=True) 

print "-- gdf_stations_json.shape : ", gdf_stations_json.shape

-- gdf_stations_json.shape :  (13039, 28)


In [33]:
gdf_stations_json.head(7)

,NUM_DEP,NOM_COM,CD_ME_niv1_surf,CD_ME_v2,CD_STATION,INDEX_STATION,NUM_COM,codagence,ALTITUDE,PROFONDEUR_MAXI_POINT,...,fi_ma_2009,fi_ma_2010,fi_ma_2011,fi_ma_2012,fi_ma_2013,fi_ma_2014,CD_STATION_,LAT_WSG84,LONG_WSG84,geometry
NUM_COM,,,,,,,,,,,,,,,,,,,,,
01004,01,AMBERIEU-EN-BUGEY,DG149,DG149,06758X0052/HY,34,01004,AERM&C,310.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,06758X0052/HY,5.364695,45.971238,POINT (5.364695114040001 45.9712376427)
01006,01,AMBLEON,DG149,DG149,07007X0001/006A,35,01006,AERM&C,420.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,07007X0001/006A,5.579207,45.753627,POINT (5.57920725651 45.7536265801)
01007,01,AMBRONAY,DG389,NaN,06754X0040/007A,36,01007,AERM&C,243.0,12.5,...,NaN,NaN,NaN,NaN,NaN,NaN,06754X0040/007A,5.331441,45.999432,POINT (5.3314414812 45.99943183689999)
01007,01,AMBRONAY,DG389,DG389,06754X0065/P2,37,01007,AERM&C,243.0,21,...,NaN,NaN,NaN,NaN,oui,NaN,06754X0065/P2,5.314578,45.994951,POINT (5.31457812135 45.9949511342)
01007,01,AMBRONAY,DG389,DG389,06754X0071/P00060,38,01007,AERM&C,243.0,NaN,...,oui,NaN,NaN,NaN,NaN,NaN,06754X0071/P00060,5.344275,45.984844,POINT (5.34427492816 45.9848443524)
01008,01,AMBUTRIX,DG240,DG389,06758X0059/HY,39,01008,AERM&C,276.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,06758X0059/HY,5.351481,45.939370,POINT (5.351481423380001 45.93936973149999)
01010,01,ANGLEFORT,DG148,DG148,07011X0028/HY,40,01010,AERM&C,770.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,07011X0028/HY,5.790001,45.936740,POINT (5.79000107015 45.9367399642)


In [34]:
print list(gdf_stations_json.columns)

[u'NUM_DEP', u'NOM_COM', u'CD_ME_niv1_surf', u'CD_ME_v2', u'CD_STATION', u'INDEX_STATION', u'NUM_COM', u'codagence', u'ALTITUDE', u'PROFONDEUR_MAXI_POINT', u'reseau2009', u'reseau2010', u'reseau2011', u'reseau2012', u'reseau2013', u'reseau2014', u'fi_ma_2007', u'fi_ma_2008', u'fi_ma_2009', u'fi_ma_2010', u'fi_ma_2011', u'fi_ma_2012', u'fi_ma_2013', u'fi_ma_2014', u'CD_STATION_', u'LAT_WSG84', u'LONG_WSG84', 'geometry']


In [35]:
### drop useless columns for geographic purposes

cols_to_drop = [
    #u'NUM_COM', 
    #u'codagence', 
    #u'ALTITUDE', 
    #u'PROFONDEUR_MAXI_POINT',
    u'reseau2009', u'reseau2010', u'reseau2011', u'reseau2012', u'reseau2013', u'reseau2014', 
    u'fi_ma_2007', u'fi_ma_2008', u'fi_ma_2009', u'fi_ma_2010', u'fi_ma_2011', u'fi_ma_2012', u'fi_ma_2013', u'fi_ma_2014', 
    #u'CD_STATION_', 
    #u'LAT_WSG84', 
    #u'LONG_WSG84', 
    #'geometry'
    ]
gdf_stations_json.drop(cols_to_drop, axis=1, inplace=True)


In [36]:
gdf_stations_json = gdf_stations_json.dropna( axis=0, how="all") # on empty rows


In [37]:
gdf_stations.shape

(13039, 28)

In [38]:
gdf_stations_json.head()

,NUM_DEP,NOM_COM,CD_ME_niv1_surf,CD_ME_v2,CD_STATION,INDEX_STATION,NUM_COM,codagence,ALTITUDE,PROFONDEUR_MAXI_POINT,CD_STATION_,LAT_WSG84,LONG_WSG84,geometry
NUM_COM,,,,,,,,,,,,,,
01004,01,AMBERIEU-EN-BUGEY,DG149,DG149,06758X0052/HY,34,01004,AERM&C,310.0,NaN,06758X0052/HY,5.364695,45.971238,POINT (5.364695114040001 45.9712376427)
01006,01,AMBLEON,DG149,DG149,07007X0001/006A,35,01006,AERM&C,420.0,NaN,07007X0001/006A,5.579207,45.753627,POINT (5.57920725651 45.7536265801)
01007,01,AMBRONAY,DG389,NaN,06754X0040/007A,36,01007,AERM&C,243.0,12.5,06754X0040/007A,5.331441,45.999432,POINT (5.3314414812 45.99943183689999)
01007,01,AMBRONAY,DG389,DG389,06754X0065/P2,37,01007,AERM&C,243.0,21,06754X0065/P2,5.314578,45.994951,POINT (5.31457812135 45.9949511342)
01007,01,AMBRONAY,DG389,DG389,06754X0071/P00060,38,01007,AERM&C,243.0,NaN,06754X0071/P00060,5.344275,45.984844,POINT (5.34427492816 45.9848443524)


In [39]:
#gdf_stations = gdf_stations.reset_index()
#gdf_stations.head(3)

In [40]:
### save geojson for stations


### filepath for stations_web.geojson
out_filename = "stations_web_carto.geojson"
out_filepath = os.path.join(carto_web_path, out_filename )


### overule Fiona overwrite bug
try: 
    os.remove(out_filepath)
except OSError:
    pass


#fiona.supported_drivers
gdf_stations_json.to_file( out_filepath, driver="GeoJSON" )

